In [1]:
import numpy as np
import tensorflow as tf

To create the seq2seq model in TF first we have to create placeholders as if they were variables.

The placeholders for the inputs and the targets are going to be ints because we are using the int mapping of the words and ints.

And we are going to create 2 more variables: the learning rate, to have some control on how much does the step moves and the keep_prob to control the droput rates.

In [2]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets = tf.placeholder(tf.int32, [None, None], name = 'target')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

In order to process the targets, they will be processed in batches, to do so, we have to use the token `<SOS>` in front of each batch and concate it with all the answers in the batch except the last column because it is not needed for the decoder.

In [3]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

# Encoder Layer

In [7]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    """
    Encoder layer
    Parameters:
      rnn_inputs - model inputs.
      rnn_size - number of tensors of the encoder rnn layer. 
      num_layers - numnber of layers.
      keep_prob - Dropout rate.
      sequence_length - Batch_size
    """
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state
 